In [1]:
import torch
import gc

gc.collect()

torch.cuda.empty_cache()

In [2]:
from unsloth import FastLanguageModel
max_seq_length = 2048  # 任意の値を選択可能。RoPEスケーリングは内部で自動的にサポート
dtype = torch.bfloat16  # 自動検出の場合はNone。Tesla T4、V100の場合はFloat16、Ampere以降の場合はBfloat16
load_in_4bit = True  # メモリ使用量を削減するために4ビット量子化を使用。Falseも可能

# 4倍高速なダウンロードとOOMの回避のためにサポートされている4ビット事前量子化モデル
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",  # 新しいMistral v3は2倍高速
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 15兆トークンモデルは2倍高速
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",  # Phi-3は2倍高速
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",  # Gemmaは2.2倍高速
]  # その他のモデルについては、https://huggingface.co/unsloth をご覧ください

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token="hf_...",  # meta-llama/Llama-2-7b-hfのようなゲート付きモデルを使用する場合は、これを使用
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: GRID A100-10C. Max memory: 9.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


RuntimeError: CUDA driver error: operation not supported

In [ ]:
!python3 -V

In [ ]:
!pip3 list